In [1]:
!pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 56.1 MB/s eta 0:00:00


In [ ]:
# Zadanie 1

!pip install pulp

import pulp as pl
import numpy as np
from scipy.stats import multivariate_t

# Parameters of the Student's t-distribution
mu = np.array([9, 8, 7, 6])
Sigma = np.array([[16, -2, -1, -3],
                  [-2, 9, -4, -1],
                  [-1, -4, 4, 1],
                  [-3, -1, 1, 1]])
v = 4  # Degrees of freedom

# Alpha and Beta values for clipping
alpha = 5
beta = 12

# Number of scenarios
n_scenarios = 2

# Seed for reproducibility
np.random.seed(0)

# Generate random samples from multivariate t-distribution
t_dist = multivariate_t(loc=mu, shape=Sigma, df=v)
samples = t_dist.rvs(size=n_scenarios)

# Clip the values
scenarios = np.clip(samples, alpha, beta)

# Print generated scenarios
print("Generated Price Scenarios:")
print(scenarios)

print("==============================================================================")

# Define the problem
model = pl.LpProblem("Maximize_Profit", pl.LpMaximize)

# Products and months
products = ['P1', 'P2', 'P3', 'P4']
months = ['Jan', 'Feb', 'Mar']

# Income from product sales in PLN/piece
product_income = {'P1': 8.5796, 'P2': 8.344, 'P3': 7.6828, 'P4': 6.4757}

# Time available per month (in hours)
days_per_month = 24
shifts_per_day = 2
hours_per_shift = 8
hours_per_month = days_per_month * shifts_per_day * hours_per_shift

# Initial stock and storage cost
initial_stock = 50
storage_cost_per_unit_per_month = 1
storage_capacity = 200
end_stock = 50

# Production times required per product in hours
production_times = {
    'P1': {'Grinding': 0.4, 'VerticalDrilling': 0.2, 'HorizontalDrilling': 0.1, 'Milling': 0.06, 'Lathing': 0},
    'P2': {'Grinding': 0.6, 'VerticalDrilling': 0.1, 'HorizontalDrilling': 0, 'Milling': 0.04, 'Lathing': 0.05},
    'P3': {'Grinding': 0, 'VerticalDrilling': 0, 'HorizontalDrilling': 0.7, 'Milling': 0, 'Lathing': 0.02},
    'P4': {'Grinding': 0, 'VerticalDrilling': 0.6, 'HorizontalDrilling': 0, 'Milling': 0.05, 'Lathing': 0}
}

# Market limits for sales of Products
market_limits = {
    "P1": {"Jan": 200, "Feb": 300, "Mar": 0},
    "P2": {"Jan": 0, "Feb": 100, "Mar": 300},
    "P3": {"Jan": 100, "Feb": 200, "Mar": 100},
    "P4": {"Jan": 200, "Feb": 200, "Mar": 200},
}

# Machines available
machines = {"Grinding": 4, "VerticalDrilling": 2, "HorizontalDrilling": 3, "Milling": 1, "Lathing": 1}

# Variables for production, sales, and storage of products each month
production_vars = pl.LpVariable.dicts("Production", ((product, month) for product in products for month in months), lowBound=0, cat='Integer')
sales_vars = pl.LpVariable.dicts("Sales", ((product, month) for product in products for month in months), lowBound=0, cat='Integer')
storage_vars = pl.LpVariable.dicts("Storage", ((product, month) for product in products for month in months), lowBound=0, upBound=storage_capacity, cat='Integer')

# Constrain for P4 sales based on P1 and P2 sales
for month in months:
  model += sales_vars['P4', month] >= sales_vars['P1', month] + sales_vars['P2', month]

# Inventory and sales constraints
for product in products:
    model += storage_vars[product, 'Jan'] == initial_stock + production_vars[product, 'Jan'] - sales_vars[product, 'Jan']  # for January
    for i in range(1, len(months)):
        month = months[i]
        prev_month = months[i-1]
        model += storage_vars[product, month] == storage_vars[product, prev_month] + production_vars[product, month] - sales_vars[product, month]  # for rest of the months
    model += storage_vars[product, 'Mar'] == end_stock  # Ending stock

# Machine time constraints for each product and month
for month in months:
    total_production_time = sum(production_vars[product, month] * production_times[product][machine]
                                for product in products
                                for machine in machines)
    model += total_production_time <= hours_per_month, f"Global_production_time_{month}"

# Market sales constraints
for product in products:
    for month in months:
        model += sales_vars[product, month] <= market_limits[product][month], f"Max_sales_{product}_{month}"
        model += sales_vars[product, month] <= production_vars[product, month] + (storage_vars[product, months[months.index(month) - 1]] if month != 'Jan' else initial_stock)


# Objective function: Maximize profit from sales minus storage costs
model += pl.lpSum([sales_vars[product, month] * product_income[product] - storage_vars[product, month] * storage_cost_per_unit_per_month
                   for product in products for month in months]), "Total_Profit"

# Solve the model
model.solve()

# Check the status of the solution and print the results
print("Status:", pl.LpStatus[model.status])
if pl.LpStatus[model.status] == 'Optimal':
    print("Total Profit:", pl.value(model.objective))

    # Printing headers
    print(f"{'Product':<10} {'Month':<10} {'Production':<12} {'Sales':<10} {'Storage':<10}")

    # Display the results in tabular form
    for product in products:
        for month in months:
            production = production_vars[product, month].varValue
            sales = sales_vars[product, month].varValue
            storage = storage_vars[product, month].varValue
            print(f"{product:<10} {month:<10} {production:<12} {sales:<10} {storage:<10}")

    print("\nProduction Time Usage per Product and Month:")
    for month in months:
        print(f"\nMonth: {month}")
        for product in products:
            total_production_time = sum(
                production_vars[product, month].varValue * production_times[product].get(machine, 0)
                for machine in machines.keys()
            )
            print(f"  Product: {product}, Total Production Time: {total_production_time} hours")

else:
    print("No optimal solution was found.")

In [ ]:
# Zadanie 2

import pulp as pl
import numpy as np
from scipy.stats import multivariate_t
import matplotlib.pyplot as plt

# Parameters
mu = np.array([9, 8, 7, 6])
Sigma = np.array([[16, -2, -1, -3], [-2, 9, -4, -1], [-1, -4, 4, 1], [-3, -1, 1, 1]])
v = 4
alpha, beta = 5, 12
n_scenarios = 35
np.random.seed(27)
t_dist = multivariate_t(loc=mu, shape=Sigma, df=v)
samples = t_dist.rvs(size=n_scenarios)
scenarios = np.clip(samples, alpha, beta)

# Problem Definition
model = pl.LpProblem("Maximize_Profit", pl.LpMaximize)

# Products and months
products = ['P1', 'P2', 'P3', 'P4']
months = ['Jan', 'Feb', 'Mar']

# Time available per month (in hours)
days_per_month = 24
shifts_per_day = 2
hours_per_shift = 8
hours_per_month = days_per_month * shifts_per_day * hours_per_shift

# Initial stock and storage cost
initial_stock = 50
storage_cost_per_unit_per_month = 1
storage_capacity = 200
end_stock = 50

# Production times required per product in hours
production_times = {
    'P1': {'Grinding': 0.4, 'VerticalDrilling': 0.2, 'HorizontalDrilling': 0.1, 'Milling': 0.06, 'Lathing': 0},
    'P2': {'Grinding': 0.6, 'VerticalDrilling': 0.1, 'HorizontalDrilling': 0, 'Milling': 0.04, 'Lathing': 0.05},
    'P3': {'Grinding': 0, 'VerticalDrilling': 0, 'HorizontalDrilling': 0.7, 'Milling': 0, 'Lathing': 0.02},
    'P4': {'Grinding': 0, 'VerticalDrilling': 0.6, 'HorizontalDrilling': 0, 'Milling': 0.05, 'Lathing': 0}
}

# Market limits for sales of Products
market_limits = {
    "P1": {"Jan": 200, "Feb": 300, "Mar": 0},
    "P2": {"Jan": 0, "Feb": 100, "Mar": 300},
    "P3": {"Jan": 500, "Feb": 500, "Mar": 500},
    "P4": {"Jan": 200, "Feb": 200, "Mar": 200},
}

# Machines available
machines = {"Grinding": 4, "VerticalDrilling": 2, "HorizontalDrilling": 3, "Milling": 1, "Lathing": 1}

# Decision Variables
production_vars = pl.LpVariable.dicts("Production", ((product, month) for product in products for month in months), lowBound=0, cat='Integer')
sales_vars = pl.LpVariable.dicts("Sales", ((product, month) for product in products for month in months), lowBound=0, cat='Integer')
storage_vars = pl.LpVariable.dicts("Storage", ((product, month) for product in products for month in months), lowBound=0, upBound=200, cat='Integer')

# Profit variables for each scenario
profit_vars = [pl.lpSum(sales_vars[product, month] * scenarios[t][list(products).index(product)] for product in products for month in months) - pl.lpSum(storage_vars[product, month] * storage_cost_per_unit_per_month for product in products for month in months) for t in range(n_scenarios)]

# Auxiliary variables for Gini differences
d_vars = pl.LpVariable.dicts("D", ((t1, t2) for t1 in range(n_scenarios) for t2 in range(t1+1, n_scenarios)), lowBound=0, cat='Continuous')

# Constrain for P4 sales based on P1 and P2 sales
for month in months:
    model += sales_vars['P4', month] >= sales_vars['P1', month] + sales_vars['P2', month], f"P4_sales_constraint_{month}"

# Inventory and sales constraints
for product in products:
    model += storage_vars[product, 'Jan'] == initial_stock + production_vars[product, 'Jan'] - sales_vars[product, 'Jan']  # for January
    for i in range(1, len(months)):
        month = months[i]
        prev_month = months[i-1]
        model += storage_vars[product, month] == storage_vars[product, prev_month] + production_vars[product, month] - sales_vars[product, month]  # for rest of the months
    model += storage_vars[product, 'Mar'] == end_stock  # Ending stock

# Machine time constraints for each product and month
for month in months:
    total_production_time = sum(production_vars[product, month] * production_times[product][machine]
                                for product in products
                                for machine in machines)
    model += total_production_time <= hours_per_month, f"Global_production_time_{month}"

# Market sales constraints
for product in products:
    for month in months:
        model += sales_vars[product, month] <= market_limits[product][month], f"Max_sales_{product}_{month}"
        model += sales_vars[product, month] <= production_vars[product, month] + (storage_vars[product, months[months.index(month) - 1]] if month != 'Jan' else initial_stock)


# Constraints for Gini differences
for t1 in range(n_scenarios):
    for t2 in range(t1+1, n_scenarios):
        model += d_vars[t1, t2] >= profit_vars[t1] - profit_vars[t2], f"Gini_difference_1_{t1}_{t2}"
        model += d_vars[t1, t2] >= profit_vars[t2] - profit_vars[t1], f"Gini_difference_2_{t2}_{t1}"

# Define the probability of each scenario
probabilities = np.ones(n_scenarios) / n_scenarios
print(probabilities)

# Loop over lambda_values
lambda_values = np.linspace(0, 25, num=100)
results = []

for lambda_value in lambda_values:
    objective_profit = pl.lpSum(probabilities[t] * profit_vars[t] for t in range(n_scenarios))
    objective_risk =  pl.lpSum(probabilities[t1] * probabilities[t2] * d_vars[min(t1, t2), max(t1, t2)] for t1 in range(n_scenarios) for t2 in range(n_scenarios) if t1 != t2)

    # Objective
    current_objective = objective_profit - lambda_value * objective_risk

    # Set the current objective in the model
    model.setObjective(current_objective)

    # Solve the model
    model.solve(pl.PULP_CBC_CMD(timeLimit=60))

    # Store results
    total_profit = pl.value(objective_profit)
    total_risk = pl.value(objective_risk)
    results.append((total_profit, total_risk, lambda_value))

    # Check the status of the solution and print the results
    print("Status:", pl.LpStatus[model.status])
    if pl.LpStatus[model.status] == 'Optimal':
        print("Total Profit:", pl.value(model.objective))

        # Printing headers
        print(f"{'Product':<10} {'Month':<10} {'Production':<12} {'Sales':<10} {'Storage':<10}")

        # Display the results in tabular form
        for product in products:
            for month in months:
                production = production_vars[product, month].varValue
                sales = sales_vars[product, month].varValue
                storage = storage_vars[product, month].varValue
                print(f"{product:<10} {month:<10} {production:<12} {sales:<10} {storage:<10}")

        print("\nProduction Time Usage per Product and Month:")
        for month in months:
            print(f"\nMonth: {month}")
            for product in products:
                total_production_time = sum(
                    production_vars[product, month].varValue * production_times[product].get(machine, 0)
                    for machine in machines.keys()
                )
                print(f"  Product: {product}, Total Production Time: {total_production_time} hours")

    else:
        print("No optimal solution was found.")


# Plotting the results
profits, risks, lambdas = zip(*results)
plt.figure(figsize=(10, 6))
plt.scatter(risks, profits, c=lambdas, cmap='viridis')
plt.colorbar(label='Lambda Value')
plt.title('Risk-Reward Frontier')
plt.xlabel('Total Risk')
plt.ylabel('Total Profit')
plt.grid(True)
plt.show()

def is_dominated(candidate, others):
    for other in others:
        if (other[0] >= candidate[0] and other[1] <= candidate[1]) and (other[0] > candidate[0] or other[1] < candidate[1]):
            return True
    return False

def find_pareto_solutions(results):
    sorted_by_profit = sorted(results, key=lambda x: x[0], reverse=True)  # Maximizing profit
    pareto_optimal = []
    for candidate in sorted_by_profit:
        if not is_dominated(candidate, pareto_optimal):
            pareto_optimal.append(candidate)
    return pareto_optimal

def find_extreme_pareto_solutions(results):
    pareto_optimal = []
    min_risk = float('inf')
    max_profit = float('-inf')
    min_risk_solution = None
    max_profit_solution = None

    for candidate in sorted(results, key=lambda x: x[0], reverse=True):  # Maximize profit sorting
        if not is_dominated(candidate, pareto_optimal):
            pareto_optimal.append(candidate)
            if candidate[1] < min_risk:
                min_risk = candidate[1]
                min_risk_solution = candidate
            if candidate[0] > max_profit:
                max_profit = candidate[0]
                max_profit_solution = candidate

    return pareto_optimal, min_risk_solution, max_profit_solution

# Calculate Pareto optimal solutions and the extremes
pareto_solutions, min_risk_sol, max_profit_sol = find_extreme_pareto_solutions(results)

print("Pareto Optimal Solutions:")
for solution in pareto_solutions:
    print(f"Profit: {solution[0]}, Risk: {solution[1]}, Lambda: {solution[2]}")

print("\nMinimum Risk Solution:")
print(f"Profit: {min_risk_sol[0]}, Risk: {min_risk_sol[1]}, Lambda: {min_risk_sol[2]}")

print("\nMaximum Profit Solution:")
print(f"Profit: {max_profit_sol[0]}, Risk: {max_profit_sol[1]}, Lambda: {max_profit_sol[2]}")
